In [ ]:
!pip install --ignore-installed wandb
!pip install --upgrade setuptools pip --user
!pip install --ignore-installed PyYAML
!pip install --ignore-installed roboflow

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
from roboflow import Roboflow

In [ ]:
import sys
import torch
from PIL import Image
print(f"Python version: {sys.version}, {sys.version_info} ")
print(f"Pytorch version: {torch.__version__} ")
%cd yolov7
print()
!ls

In [ ]:
!nvidia-smi

In [ ]:
!git clone https://github.com/ncallahanml/yolov7.git
!git checkout u7

In [ ]:
weight_strs = ['yolov7_training.pt','yolov7-e6_training.pt','yolov7-w6_training.pt','yolov7x-seg.pt']
assert os.getcwd().endswith('yolov7/')
for weight_str in weight_strs:
    if not weight_str in os.listdir():
        loc = f'https://github.com/WongKinYiu/yolov7/releases/download/v0.1/{weight_str}'
        !curl -L $loc -o $weight_str 
        print(f'{weight_str} model weights finished downloading')
    else:
        print(f'{weight_str} model weights available')

In [5]:
DATA_YAML = 'data.yaml'
NAME = 'segmentation_test'
WEIGHTS = 'yolov7-seg.pt'
P6 = False
DISTRIBUTED =True
EPOCHS = 1200
IMG_SIZE = 1280

TRAIN_FILE = 'train_aux.py' if P6 else 'train.py'
LEAD = '-m torch.distributed.launch --nproc_per_node 4 --master_port 9527 ' if DISTRIBUTED else ''
DEVICE = '0,1,2,3' if DISTRIBUTED else '0'

train_str = f"!python {LEAD}{TRAIN_FILE} --workers 8 --device {DEVICE} --batch-size 8 --data data/{DATA_YAML} --img {IMG_SIZE} --cfg cfg/training/{WEIGHTS.replace('.pt','.yaml').replace('_training','')} --weights {WEIGHTS} --name {NAME} --hyp data/hyp.scratch.custom.yaml --epochs {EPOCHS}"
print(train_str)

!python -m torch.distributed.launch --nproc_per_node 4 --master_port 9527 train.py --workers 8 --device 0,1,2,3 --batch-size 8 --data data/yolov14.yaml --img 1280 --cfg cfg/training/yolov7x-seg.yaml --weights yolov7x-seg.pt --name segmentation_test --hyp data/hyp.scratch.custom.yaml --epochs 1200


In [ ]:
os.system(train_str[1:])

In [ ]:
!python -m torch.distributed.launch --nproc_per_node 4 --master_port 9527 train.py --workers 8 --device 0,1,2,3 --batch-size 8 --data data/yolov14.yaml --img 640 --cfg cfg/training/yolov7x.yaml --weights yolov7x_training.pt --name yolov7-v14_7x --hyp data/hyp.scratch.custom.custom.yaml --epochs 800

In [ ]:
rf = Roboflow(api_key='')
project = rf.workspace("tid-detection").project("ionospheresegmentation")
dataset = project.version(2).download("yolov7")

In [ ]:
!pip install -r requirements.txt
!pip install setuptools==59.5.0
!pip install torchvision==0.11.3+cu111 -f https://download.pytorch.org/whl/cu111/torch_stable.html

In [ ]:
!python detect.py --weights ./yolov7.pt --conf 0.25 --img-size 640 --source inference/images/horses.jpg